In [1]:
!pip install marshmallow-jsonapi

    100% |████████████████████████████████| 51kB 8.5MB/s ta 0:00:01


In [17]:
from marshmallow_jsonapi import Schema, fields
class Compound():
    def __init__(self, objectId, chemsys, elements, nelements, pretty_formula, formula_anonymous, material_id):
        self.chemsys = chemsys
        self.elements = elements
        self.nelements = nelements
        self.pretty_formula = pretty_formula
        self.formula_anonymous = formula_anonymous
        self.material_id = material_id
        self.id = objectId

class CompoundSchema(Schema):
    chemsys = fields.Str()
    elements = fields.List(fields.String)
    nelements = fields.Int()
    pretty_formula = fields.Str()
    formula_anonymous = fields.Str()
    material_id = fields.Str()
    id = fields.Str()
    
    class Meta:
        type_ = "compound"
        strict = True

compound_schema = CompoundSchema()

In [18]:
compound = Compound("5c4ddc4448c81a99e54c9ec0", "O-Ti",['O', 'Ti'],3,"asdf","gh","134adsf")

In [19]:
CompoundSchema().dump(compound)

MarshalResult(data={'data': {'type': 'compound', 'attributes': {'material_id': '134adsf', 'chemsys': 'O-Ti', 'nelements': 3, 'elements': ['O', 'Ti'], 'formula_anonymous': 'gh', 'pretty_formula': 'asdf'}, 'id': '5c4ddc4448c81a99e54c9ec0'}}, errors={})